In [29]:
#Import Libraries Needed
import pandas as pd
import geopandas as gpd
import panel as pn
pn.extension('plotly',comms='ipywidgets')
import param
import hvplot.pandas
import plotly.express as px
from urllib.request import urlopen
import requests
from PIL import Image, ImageOps

In [30]:
css = '''
.bk.panel-widget-box {
  background: #f0f0f0;
  border-radius: 5px;
  border: 1px black solid;
  font-size: 24px;
}
'''

pn.extension(raw_css=[css])

In [31]:
#Import DataFrames
import json
with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
    counties = json.load(response)
target_states = ['48']
counties['features'] = [f for f in counties['features'] if f['properties']['STATE'] in target_states]
address = ("Data/District_Loss/district.csv")
dis_df = pd.read_csv(address)

#Create a new dataframe with columns based off the differences of 2021 National Center on Education Statistics and
#2019 National Center on Education Statistics, divided by the 2019 data.
dis_copy = dis_df.copy()
countynames = dis_df['County Name'].unique()
countynames.sort()
fips = dis_df['County #'].unique()
fips.sort()
dis_copy = dis_copy.groupby('County Name').mean()
dis_copy = dis_copy.drop(['District #'], axis = 1)
dis_copy['County #'] = fips
dis_copy['County Name'] = countynames
dis_copy= dis_copy.reset_index(drop=True)
dis_copy['Full-Time Equivalent(FTE) Teachers Difference (2021-2019)'] = (dis_copy['FTE Teachers 2020-2021']-dis_copy['FTE Teachers 2018-2019'])/dis_copy['FTE Teachers 2018-2019']
dis_copy['Sum of Full-Time Equivalent(FTE) Teachers and Staffs Difference (2021-2019)'] = (dis_copy['Total Staff 2020-2021']-dis_copy['Total Staff 2018-2019'])/dis_copy['Total Staff 2018-2019']
dis_copy['Total Number of Students Difference (2021-2019)'] = (dis_copy['Total Students 2020-2021']-dis_copy['Total Students 2018-2019'])/dis_copy['Total Students 2018-2019']

C:\Users\Daniel Payan\AppData\Local\Temp\ipykernel_18620\3196620513.py:17: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.



In [32]:
dis_copy = dis_copy.round(2)

In [33]:
#Import dataframes
address = ("Data/District_Loss/txcountyemploy.xlsx")
employ_df = pd.read_excel(address)

#Create a new dataframes with columns based on the differences of the 2021 Bureau of Labor Statistics Data and 
#2019 Bureau of Labor Statistics Data, divided by the 2019 data.
employ_df['NAME'] = employ_df['NAME'].str.replace(", TX","")
employ_df['YEAR']=employ_df['YEAR'].values.astype(str)
employ_df['YEAR'] = employ_df['YEAR'].str.replace("'","")
df_19 = employ_df[employ_df['YEAR'] == '2019']
df_20 = employ_df[employ_df['YEAR'] == '2020']
df_21 = employ_df[employ_df['YEAR'] == '2021']
df_20 =df_20.reset_index()
df_21=df_21.reset_index()
diff_df = pd.DataFrame()
diff_df = df_19[['NAME','FIPS']]
diff_df['Civilian Labor Force Difference (2021-2019)'] = (df_21['LABOR_FORCE']-df_19['LABOR_FORCE'])/df_19['LABOR_FORCE']
diff_df['Employed Persons Difference (2021-2019)']= (df_21['EMPLOYED']-df_19['EMPLOYED'])/df_19['EMPLOYED']
diff_df['Umemployed Persons Difference (2021-2019)']= (df_21['UNEMPLOYED']-df_19['UNEMPLOYED'])/df_19['UNEMPLOYED']
diff_df['Unemployment Rate Difference (2021-2019)']= (df_21['UNEMPLOYMENT_PCT']-df_19['UNEMPLOYMENT_PCT'])/df_19['UNEMPLOYMENT_PCT']

D:\anaconda\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning:

Workbook contains no default style, apply openpyxl's default

C:\Users\Daniel Payan\AppData\Local\Temp\ipykernel_18620\1615083697.py:17: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\Daniel Payan\AppData\Local\Temp\ipykernel_18620\1615083697.py:18: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\Daniel Payan\AppData\Local\Temp\ipykernel_18620\1615083697.py:19: SettingWithCopyWarning:


A value is trying to be set 

In [34]:
diff_df = diff_df.round(2)
diff_df.rename(columns={'NAME':'County Name','FIPS':'County #'},inplace=True)

In [35]:
#Import dataframes
address = ("Data/District_Loss/DATA_Texas_District_with_Labels.csv")
disloss_df = pd.read_csv(address)
address = ("Data/District_Loss/district.csv")
dis_df = pd.read_csv(address)

#Creates a dataframe based on the Texas Education Agency data based on the differences of
#2021 and 2019 data, divided by 2019 data.
disloss_df['County #'] = disloss_df['County #'].astype(int)
countynames = dis_df['County Name'].unique()
countynames.sort()
fips = dis_df['County #'].unique()
fips.sort()

discopy_df = pd.DataFrame()
discopy_df['County #'] = disloss_df['County #']
discopy_df['Average Score Difference(%) in Math (2021-2019)'] = (disloss_df['AVG_Math_2021']-disloss_df['AVG_Math_2019'])/disloss_df['AVG_Math_2019']
discopy_df['Average Score Difference(%) in Reading (2021-2019)'] = (disloss_df['AVG_Reading_2021']-disloss_df['AVG_Reading_2019'])/disloss_df['AVG_Reading_2019']

discopy_df = discopy_df.groupby('County #').mean()
discopy_df['County #'] = fips
discopy_df['County Name'] = countynames
discopy_df = discopy_df.reset_index(drop=True)

In [36]:
discopy_df = discopy_df.round(2)

In [37]:
discopy_df.drop(columns='County Name',inplace=True)
dis_copy.drop(columns='County Name',inplace=True)
comp_df = discopy_df.merge(diff_df,on='County #')
comp_df = comp_df.merge(dis_copy,on='County #')

In [38]:
comp_df.columns

Index(['Average Score Difference(%) in Math (2021-2019)',
       'Average Score Difference(%) in Reading (2021-2019)', 'County #',
       'County Name', 'Civilian Labor Force Difference (2021-2019)',
       'Employed Persons Difference (2021-2019)',
       'Umemployed Persons Difference (2021-2019)',
       'Unemployment Rate Difference (2021-2019)', 'FTE Teachers 2018-2019',
       'Total Staff 2018-2019', 'Total Students 2018-2019',
       'FTE Teachers 2020-2021', 'Total Staff 2020-2021',
       'Total Students 2020-2021',
       'Full-Time Equivalent(FTE) Teachers Difference (2021-2019)',
       'Sum of Full-Time Equivalent(FTE) Teachers and Staffs Difference (2021-2019)',
       'Total Number of Students Difference (2021-2019)'],
      dtype='object')

In [39]:
comp_df.rename(columns={'Full-Time Equivalent(FTE) Teachers Difference (2021-2019)':'Number of Teachers',
                          'Sum of Full-Time Equivalent(FTE) Teachers and Staffs Difference (2021-2019)':'Number of Overall Staff',
                          'Total Number of Students Difference (2021-2019)': 'Number of Students',
                          'Civilian Labor Force Difference (2021-2019)':'Labor Force',
                          'Average Score Difference(%) in Math (2021-2019)': 'STAAR Score in Math',
                          'Average Score Difference(%) in Reading (2021-2019)': 'STAAR Score in Reading',},inplace=True)

In [40]:
comp_df['Number of Teachers'] = comp_df['Number of Teachers']*100
comp_df['Number of Overall Staff'] = comp_df['Number of Overall Staff']*100
comp_df['Number of Students'] = comp_df['Number of Students']*100
comp_df['Labor Force'] = comp_df['Labor Force']*100
comp_df['STAAR Score in Math'] = comp_df['STAAR Score in Math']*100
comp_df['STAAR Score in Reading'] = comp_df['STAAR Score in Reading']*100
comp_df['County Name'] = comp_df['County Name'].str[:-7]

# Texas County Level Map

In [41]:
comp_df.dtypes

STAAR Score in Math                          float64
STAAR Score in Reading                       float64
County #                                       int64
County Name                                   object
Labor Force                                  float64
Employed Persons Difference (2021-2019)      float64
Umemployed Persons Difference (2021-2019)    float64
Unemployment Rate Difference (2021-2019)     float64
FTE Teachers 2018-2019                       float64
Total Staff 2018-2019                        float64
Total Students 2018-2019                     float64
FTE Teachers 2020-2021                       float64
Total Staff 2020-2021                        float64
Total Students 2020-2021                     float64
Number of Teachers                           float64
Number of Overall Staff                      float64
Number of Students                           float64
dtype: object

In [42]:
comp_df['Number of Teachers'] = comp_df['Number of Teachers'].astype(int)
comp_df['STAAR Score in Math'] = comp_df['STAAR Score in Math'].astype(int)
comp_df['STAAR Score in Reading'] = comp_df['STAAR Score in Reading'].astype(int)
comp_df['Labor Force'] = comp_df['Labor Force'].astype(int)
comp_df['Number of Overall Staff'] = comp_df['Number of Overall Staff'].astype(int)
comp_df['Number of Students'] = comp_df['Number of Students'].astype(int)

In [43]:
comp_df

,STAAR Score in Math,STAAR Score in Reading,County #,County Name,Labor Force,Employed Persons Difference (2021-2019),Umemployed Persons Difference (2021-2019),Unemployment Rate Difference (2021-2019),FTE Teachers 2018-2019,Total Staff 2018-2019,Total Students 2018-2019,FTE Teachers 2020-2021,Total Staff 2020-2021,Total Students 2020-2021,Number of Teachers,Number of Overall Staff,Number of Students
0,-2,0,48001,Anderson,0,-0.02,0.65,0.66,95.96,185.88,1186.57,96.27,196.87,1137.29,0,6,-4
1,-2,1,48003,Andrews,-8,-0.12,1.35,1.57,264.19,508.27,4319.00,287.18,559.88,4128.00,9,10,-4
2,-2,0,48005,Angelina,1,-0.01,0.64,0.62,180.98,380.43,2458.14,183.47,390.04,2318.86,1,3,-6
3,-4,0,48007,Aransas,1,-0.02,0.71,0.70,214.93,467.53,2829.00,217.46,452.82,3010.00,1,-3,6
4,-2,1,48009,Archer,0,-0.01,0.49,0.50,51.93,94.00,652.00,54.22,97.79,668.33,4,4,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
248,-2,0,48499,Wood,3,0.01,0.48,0.44,82.95,162.94,1042.17,87.46,170.28,1012.83,5,5,-3
249,-2,0,48501,Yoakum,-11,-0.16,1.40,1.69,79.56,159.07,1091.00,81.32,167.53,1034.50,2,5,-5
250,-1,1,48503,Young,2,0.00,0.55,0.50,86.54,169.87,1086.33,88.00,171.08,1065.33,2,1,-2
251,-6,-2,48505,Zapata,-11,-0.17,1.14,1.42,226.78,490.74,3522.00,227.95,519.55,3486.00,1,6,-1


In [44]:
comp_df.to_csv('Data/CountyLevelstats.csv')

In [45]:
#Creates a county level map of Texas based on the Texas Education Agency Data
input_list = ['Number of Teachers',
              'Number of Overall Staff',
              'Number of Students',
              'Labor Force',
              'STAAR Score in Math',
              'STAAR Score in Reading',]
class MapDashboard(param.Parameterized):
    _  = param.Selector(input_list,default=input_list[0])
    @param.depends('_')
    def get_map(self):
        data = comp_df[['County Name','County #',self._]]
        fig = px.choropleth_mapbox(data, geojson=counties, locations='County #', color= self._,
        color_continuous_scale='balance_r',
        range_color=(-10,10),
       # title ='Work,
        center = {'lat':31.9686, 'lon': -99.9018},
        zoom = 4.72,# labels={'Number of Students':'Percent Change in Number of Students is: '},
        height=585, width=525,
        mapbox_style = "white-bg",       
        custom_data = [data['County Name'],self._]
        #hover_name = 'County Name',
        #hover_data={'County #':False}
        )
        hovertemp = ''
        hovertemp = '%{customdata[0]} County<br>'                   
        hovertemp += '%{customdata[1]}% change in ' + self._
        fig.update_traces(hovertemplate=hovertemp)
        fig.update_geos(fitbounds='locations')
        fig.update_layout(hoverlabel=dict(bgcolor='black',font_size=24,font_family="Times New Roman"),
        font=dict(family='times new roman',
        size=24,color='black'),
       # title_x=.96,
       # title_y=.97,                  
        margin={"r":10,"t":0,"l":10,"b":0})
        fig.update_layout(coloraxis_colorbar=dict(tickvals=[-8,-4,0,4,8],ticktext=['-10','-5','0','5','10'])),
        fig.update_coloraxes(cmid=0,colorbar_orientation='h',colorbar_y=-.2,colorbar_title_font_family='Times New Roman',
                             colorbar_title_font_size=20,colorbar_title_side='bottom', colorbar_thickness=4)
        return fig
    def get_description(self):
        if self._ == 'STAAR Score in Math':
            description = pn.pane.HTML('The percent difference in Math STAAR scores between 2019 and 2021 for a county.',
                            style={'color':'black','font-size':'20px','width':'125%',
                                  'text-align':'left',
                                  'font-family':'Times New Roman'})
        elif self._ == 'STAAR Score in Reading':
            description = pn.pane.HTML('The percent difference in Reading STAAR scores between 2019 and 2021 for a county.',
                            style={'color':'black','font-size':'20px','width':'125%',
                                  'text-align':'left',
                                  'font-family':'Times New Roman'})
        elif self._ == 'Labor Force':
            description = pn.pane.HTML('The percent difference in labor force between 2019 and 2021 for a county.',
                            style={'color':'black','font-size':'20px','width':'125%',
                                  'text-align':'left',
                                  'font-family':'Times New Roman'})
        elif self._ == 'Number of Teachers':
            description = pn.pane.HTML('The percent difference in Full-Time Teachers between 2019 and 2021 for a county.',
                            style={'color':'black','font-size':'20px','width':'125%',
                                  'text-align':'left',
                                  'font-family':'Times New Roman'})
        elif self._ == 'Number of Overall Staff':
            description = pn.pane.HTML('The percent difference in staff between 2019 and 2021 for a county.',
                            style={'color':'black','font-size':'20px','width':'125%',
                                  'text-align':'left',
                                  'font-family':'Times New Roman'})
        elif self._== 'Number of Students':
            description = pn.pane.HTML('The percent difference in students between 2019 and 2021 for a county.',
                            style={'color':'black','font-size':'20px','width':'125%',
                                  'text-align':'left',
                                  'font-family':'Times New Roman'})
        return description
    def panel(self):
        return pn.Row(
            pn.Column(
            pn.pane.HTML("""<h1>Select A Variable To View At The County Level</h1>""",
                              style={'color':'black','font-size':'14px','width':'100%',
                                     'text-align':'center',
                                     'font-family':'Times New Roman'}),
            pn.Param(self,width=550,background='#FFFFFF'),
            pn.Spacer(height=50),
            pn.pane.HTML("""<h1>What does this variable show us?</h1>""",
                            style={'color':'black','font-size':'14px','width':'100%',
                                  'text-align':'center',
                                  'font-family':'Times New Roman'}),
            pn.Spacer(height= 5),
            self.get_description
            #pn.Param(self, width=500,background='#FFFFFF'),
            ),
            pn.Spacer(width=150),
            self.get_map
        )
dashboard = MapDashboard(name='')
map_component1 = dashboard.panel()
map_component1

BokehModel(combine_events=True, render_bundle={'docs_json': {'fcc02be2-4a72-491d-b1d6-4f55049d0ef5': {'defs': …

# Heatmap & Histogram

In [46]:
#Import dataframe
address = ("Data/District_Loss/loss_df.csv")
loss_df = pd.read_csv(address)
loss_df = loss_df.round(2)

In [47]:
loss_df.rename(columns = {'% LOSS Math':'Math','% LOSS Reading':'Reading'},inplace=True)

In [48]:
loss_df['Math'] = loss_df['Math']*-100
loss_df['Reading'] = loss_df['Reading']*100

In [49]:
loss_df

,Unnamed: 0,Agency Name,State,NCES Agency ID,County #,Total Operational Public Schools 2018-2019,Total Public Schools 2018-2019,Total Students 2018-2019,FTE Teachers 2018-2019,Total Staff 2018-2019,...,% On Campus 09/28/20,% On Campus 10/30/20,% On Campus 01/29/21,AVG_Math_2019,AVG_Reading_2019,AVG_Math_2021,AVG_Reading_2021,Math,Reading,% LOSS All
0,0,A W BROWN LEADERSHIP ACADEMY,Texas,4800095.0,48113.0,2.0,2.0,2084.0,115.60,218.40,...,0.41,0.40,0.38,0.84,0.88,0.81,0.88,-3.0,-0.0,0.01
1,1,A+ ACADEMY,Texas,4800203.0,48113.0,2.0,2.0,1409.0,86.58,192.28,...,0.36,0.55,0.59,0.85,0.87,0.80,0.86,-4.0,1.0,0.03
2,2,A+ UNLIMITED POTENTIAL,TEXAS,4801453.0,48201.0,2.0,2.0,180.0,7.67,10.34,...,0.15,0.38,0.47,0.42,0.45,0.40,0.45,-2.0,-0.0,0.01
3,3,ABBOTT ISD,Texas,4807380.0,48217.0,1.0,2.0,277.0,24.25,40.00,...,0.92,0.97,0.99,0.92,0.94,0.90,0.94,-2.0,-0.0,0.01
4,4,ABERNATHY ISD,Texas,4807410.0,48189.0,3.0,4.0,780.0,68.00,124.97,...,0.94,0.92,0.99,0.87,0.91,0.86,0.92,-1.0,-1.0,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1160,1160,YORKTOWN ISD,Texas,4846650.0,48123.0,3.0,3.0,546.0,43.19,85.28,...,0.89,0.90,1.00,0.84,0.88,0.82,0.87,-3.0,1.0,0.02
1161,1161,YSLETA ISD,Texas,4846680.0,48141.0,61.0,61.0,41064.0,2747.52,6272.21,...,0.00,0.03,0.03,0.88,0.91,0.82,0.89,-6.0,2.0,0.04
1162,1162,ZAPATA COUNTY ISD,Texas,4846710.0,48505.0,6.0,6.0,3522.0,226.78,490.74,...,0.02,0.03,0.03,0.85,0.88,0.80,0.86,-5.0,2.0,0.04
1163,1163,ZAVALLA ISD,Texas,4846740.0,48005.0,2.0,3.0,360.0,34.35,59.76,...,0.88,0.98,0.97,0.87,0.89,0.85,0.91,-2.0,-1.0,0.00


In [50]:
#Creates a histogram based on the STAAR/Percent Loss data, displays percent loss by district count
input_list = ['Math','Reading']
class LossDashboard(param.Parameterized):
    Subject = param.Selector(input_list,default=input_list[0])
    #@param.depends('_')
    def get_map(self):
        fig = px.histogram(data_frame = loss_df, x = self.Subject,
        height=500, width=600, range_x=[-50,50],
        labels={'Math':'% Loss in Math: ','Reading':'% Loss in Reading: ','count':'# of Districts'}
        )
        fig.update_layout(xaxis_title_text='% LOSS', yaxis_title_text='Number of Districts',bargap=0.2,hoverlabel=dict(bgcolor='black',font_size=24,font_family="Times New Roman"),
        font=dict(family='times new roman',
        size=23,color='black'),
        title_x=0.55,
       # title_y=.95,
        margin={"r":10,"t":10,"l":0,"b":15})
        return fig
    def panel(self):
        return pn.Column(
            pn.Param(self, width=350, name='Normalized Difference in STAAR Scores Between 2019 and 2021, Avg. Over TX ISDs:',sizing_mode='fixed'),
            
            self.get_map
        )
dashboard = LossDashboard(name = 'Loss Dashboard')
loss_component1 = dashboard.panel()
loss_component1

BokehModel(combine_events=True, render_bundle={'docs_json': {'87a8237f-c5e9-474b-9429-f7107bd2babd': {'defs': …

In [51]:
#Import dataframe
address = ("Data/District_Loss/heat_df.csv")
heat_df = pd.read_csv(address)
#Group the dataset by demographic group
heat_df= heat_df.set_index('group')

In [52]:
heat_df['3.0'] = heat_df['3.0']*-1
heat_df['4.0'] = heat_df['4.0']*-1
heat_df['5.0'] = heat_df['5.0']*-1
heat_df['6.0'] = heat_df['6.0']*-1
heat_df['7.0'] = heat_df['7.0']*-1
heat_df['8.0'] = heat_df['8.0']*-1
heat_df['3.0'] = heat_df['3.0']*100
heat_df['4.0'] = heat_df['4.0']*100
heat_df['5.0'] = heat_df['5.0']*100
heat_df['6.0'] = heat_df['6.0']*100
heat_df['7.0'] = heat_df['7.0']*100
heat_df['8.0'] = heat_df['8.0']*100
heat_df=heat_df.round(2)
heat_df.rename(columns={'3.0':'3rd',
                      '4.0':'4th',
                      '5.0':'5th',
                      '6.0':'6th',
                      '7.0':'7th',
                      '8.0':'8th'},inplace=True)

In [53]:
heat_df.dtypes

subject     object
3rd        float64
4th        float64
5th        float64
6th        float64
7th        float64
8th        float64
dtype: object

In [54]:
heat_df

,subject,3rd,4th,5th,6th,7th,8th
group,,,,,,,
All,Math,-5.4,-4.1,-1.1,-2.3,-7.1,-0.6
Asian,Math,-5.4,-4.4,-0.9,-3.1,-7.7,-0.2
Black,Math,-5.6,-4.2,-1.1,-1.9,-6.5,-0.4
Free Lunch,Math,-5.7,-4.1,-1.1,-2.3,-6.9,-0.6
Hispanic,Math,-5.6,-4.3,-1.2,-2.4,-7.1,-0.7
Poverty,Math,-5.6,-4.2,-1.2,-2.3,-7.0,-0.7
Special Ed,Math,-4.4,-3.3,0.1,-0.6,-5.1,1.0
T1,Math,-5.5,-4.2,-1.1,-2.5,-7.1,-0.9
White,Math,-4.5,-3.2,-0.8,-1.8,-7.0,-0.1


In [55]:
#Creates a heatmap based off of the STAAR/Percent Loss Data by demographic group and grade
subject_inputs = ['Math','Reading']
class LossDashboard(param.Parameterized):
    Subject = param.Selector(subject_inputs,default = subject_inputs[0])
    @param.depends('Subject')
    def lossplot(self):
        data = heat_df[heat_df['subject'] == self.Subject]
        data = data.drop(['subject'],axis=1)
        fig = px.imshow(data, labels=dict(x='Grade',y='Demographic Group',color='Learning Loss %'),text_auto='.1f',
        height=500,width=800,
        color_continuous_scale='balance_r',
       # labels={'Learning Loss':'Learning Loss '}
        range_color=(-4,4),
        aspect='none')
        fig.update_layout(hoverlabel=dict(bgcolor='black',font_size=24,font_family="Times New Roman"),
        font=dict(family='times new roman',
        size=24,color='black'),
        title_x=.96,                
        margin={"r":10,"t":10,"l":0,"b":15})
        fig.update_coloraxes(cmid=0,colorbar_title_font_family='Times New Roman',colorbar_title_text='% LOSS',
                             colorbar_title_font_size=24,colorbar_title_side='bottom',colorbar_thickness=10)
        return fig
    def panel(self):
        return pn.Column(
            pn.Param(self, width=350, name='Normalized Difference in STAAR Scores Between 2019 and 2021 By Demographic Group and Grade:',sizing_mode = 'fixed',background='#FFFFFF'),  
            self.lossplot
        )
dashboard = LossDashboard(name = 'Loss Dashboard')
loss_component2 = dashboard.panel()
loss_component2

BokehModel(combine_events=True, render_bundle={'docs_json': {'abbfc111-837c-46aa-9ab9-760e3c948e44': {'defs': …

# Texas District Level Maps

In [56]:
tx_map = gpd.read_file("Data/ISDmaps/Districts2020to2021.geojson")
label_data = pd.read_csv("Data/District_Loss/DATA_Texas_District_v1.csv")

In [57]:
label_data["NCES_DISTR"] = label_data["NCES_DISTR"].astype(int)
label_data = label_data.sort_values("NCES_DISTR")

In [58]:
tx_map = tx_map.sort_values("NCES_DISTR")
tx_map['NCES_DISTR'] = tx_map['NCES_DISTR'].astype(int)

In [59]:
merged_df = tx_map.merge(label_data, how = 'inner', on=['NCES_DISTR','DISTRICT_N'])

In [60]:
merged_df['Full-Time Equivalent(FTE) Teachers Difference (2021-2019)'] = (merged_df['FTE Teachers 2020-2021']-merged_df['FTE Teachers 2018-2019'])/merged_df['FTE Teachers 2018-2019']
merged_df['Sum of Full-Time Equivalent(FTE) Teachers and Staffs Difference (2021-2019)'] = (merged_df['Total Staff 2020-2021']-merged_df['Total Staff 2018-2019'])/merged_df['Total Staff 2018-2019']
merged_df['Total Number of Students Difference (2021-2019)'] = (merged_df['Total Students 2020-2021']-merged_df['Total Students 2018-2019'])/merged_df['Total Students 2018-2019']

In [61]:
merged_df = merged_df.round(2)

In [62]:
merged_df.rename(columns={'% LOSS MATH':'Average Score Difference(%) in Math (2021-2019)',
                          '% LOSS READING':'Average Score Difference(%) in Reading (2021-2019)',
                          '% LOSS ALL':'Average Score Difference(%) in All (2021-2019)',
                          'Label_Math':'Learning Loss in Math: (-1) Loss (0) Expected (1) Gain',
                          'Label_Reading':'Learning Loss in Reading: (-1) Loss (0) Expected (1) Gain',
                          'Label_All':'Learning Loss in Math and Reading: (-1) Loss (0) Expected (1) Gain'},inplace=True)

In [63]:
county_dict = pd.DataFrame()
county_dict['County #'] = comp_df['County #']
county_dict['County Name'] = comp_df['County Name']
cdict = pd.Series(county_dict['County #'].values,index=county_dict['County Name']).to_dict()
ndict = pd.Series(county_dict['County Name'].values,index=county_dict['County #']).to_dict()
label_data['County Number'] = label_data['County #']
label_data.replace({'County #': ndict}, inplace=True)
label_data.rename(columns = {'County #': 'County Name','County Number':'County #'}, inplace=True)

In [64]:
label_data

,Agency Name,State,NCES_DISTR,County Name,Total Operational Public Schools 2018-2019,Total Public Schools 2018-2019,Total Students 2018-2019,FTE Teachers 2018-2019,Total Staff 2018-2019,DISTRICT_N,...,AVG_Reading_2019,AVG_Math_2021,AVG_Reading_2021,% LOSS MATH,% LOSS READING,% LOSS ALL,Label_Math,Label_Reading,Label_All,County #
254,CROSBYTON CISD,Texas,4800001,Crosby,3,4,367,37.76,67.63,54901,...,0.877215,0.808392,0.880479,-0.036684,0.003721,-0.016482,0,0,0,48107
990,SPUR ISD,Texas,4800002,Dickens,1,1,246,28.24,54.00,63903,...,0.903183,0.864715,0.939864,0.006645,0.040614,0.023629,1,1,1,48125
888,ROCKSPRINGS ISD,Texas,4800003,Edwards,1,1,306,24.03,46.55,69901,...,0.880964,0.778777,0.857982,-0.045350,-0.026088,-0.035719,-1,-1,-1,48137
94,BLACKWELL CISD,Texas,4800005,Nolan,1,2,152,22.10,42.37,177903,...,0.774391,0.837564,0.923943,0.131782,0.193121,0.162451,1,1,1,48353
51,ASPERMONT ISD,Texas,4800006,Stonewall,2,2,224,19.44,39.86,217901,...,0.892216,0.848033,0.909429,-0.017357,0.019292,0.000967,0,1,1,48433
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1161,YSLETA ISD,Texas,4846680,El Paso,61,61,41064,2747.52,6272.21,71905,...,0.905574,0.818961,0.887423,-0.068699,-0.020043,-0.044371,-1,-1,-1,48141
1162,ZAPATA COUNTY ISD,Texas,4846710,Zapata,6,6,3522,226.78,490.74,253901,...,0.882555,0.801787,0.860655,-0.057647,-0.024814,-0.041230,-1,-1,-1,48505
1163,ZAVALLA ISD,Texas,4846740,Angelina,2,3,360,34.35,59.76,3906,...,0.893549,0.848231,0.908511,-0.021311,0.016745,-0.002283,0,1,1,48005
1164,ZEPHYR ISD,Texas,4846770,Brown,1,1,218,19.48,32.99,25906,...,0.915698,0.842740,0.912110,-0.033731,-0.003918,-0.018825,0,0,0,48049


In [65]:
cdict

{'Anderson': 48001,
 'Andrews': 48003,
 'Angelina': 48005,
 'Aransas': 48007,
 'Archer': 48009,
 'Armstrong': 48011,
 'Atascosa': 48013,
 'Austin': 48015,
 'Bailey': 48017,
 'Bandera': 48019,
 'Bastrop': 48021,
 'Baylor': 48023,
 'Bee': 48025,
 'Bell': 48027,
 'Bexar': 48029,
 'Blanco': 48031,
 'Borden': 48033,
 'Bosque': 48035,
 'Bowie': 48037,
 'Brazoria': 48039,
 'Brazos': 48041,
 'Brewster': 48043,
 'Briscoe': 48045,
 'Brooks': 48047,
 'Brown': 48049,
 'Burleson': 48051,
 'Burnet': 48053,
 'Caldwell': 48055,
 'Calhoun': 48057,
 'Callahan': 48059,
 'Cameron': 48061,
 'Camp': 48063,
 'Carson': 48065,
 'Cass': 48067,
 'Castro': 48069,
 'Chambers': 48071,
 'Cherokee': 48073,
 'Childress': 48075,
 'Clay': 48077,
 'Cochran': 48079,
 'Coke': 48081,
 'Coleman': 48083,
 'Collin': 48085,
 'Collingsworth': 48087,
 'Colorado': 48089,
 'Comal': 48091,
 'Comanche': 48093,
 'Concho': 48095,
 'Cooke': 48097,
 'Coryell': 48099,
 'Cottle': 48101,
 'Crane': 48103,
 'Crockett': 48105,
 'Crosby': 48107

In [66]:
merged_df['County #']

0       48107
1       48125
2       48137
3       48353
4       48433
        ...  
1008    48141
1009    48505
1010    48005
1011    48049
1012    48131
Name: County #, Length: 1013, dtype: int64

In [67]:
merged_df.rename(columns={'Full-Time Equivalent(FTE) Teachers Difference (2021-2019)':'Number of Teachers',
                          'Sum of Full-Time Equivalent(FTE) Teachers and Staffs Difference (2021-2019)':'Number of Overall Staff',
                          'Total Number of Students Difference (2021-2019)': 'Number of Students',
                          'Average Score Difference(%) in Math (2021-2019)': 'STAAR Score in Math',
                          'Average Score Difference(%) in Reading (2021-2019)': 'STAAR Score in Reading',
                          'Learning Loss in Math: (-1) Loss (0) Expected (1) Gain': 'Learning Loss in Math',
                          'Learning Loss in Reading: (-1) Loss (0) Expected (1) Gain': 'Learning Loss in Reading',
                          'Learning Loss in Math and Reading: (-1) Loss (0) Expected (1) Gain': 'Learning Loss'},inplace=True)

In [68]:
merged_df.columns

Index(['OBJECTID', 'SDLEA10', 'NAME', 'NAME2', 'DISTRICT_N', 'DISTRICT',
       'DISTRICT_C', 'NCES_DISTR', 'COLOR', 'Shape__Area',
       ...
       'AVG_Reading_2021', 'STAAR Score in Math', 'STAAR Score in Reading',
       'Average Score Difference(%) in All (2021-2019)',
       'Learning Loss in Math', 'Learning Loss in Reading', 'Learning Loss',
       'Number of Teachers', 'Number of Overall Staff', 'Number of Students'],
      dtype='object', length=963)

In [69]:
merged_df['Number of Teachers'] = merged_df['Number of Teachers']*100
merged_df['Number of Overall Staff'] = merged_df['Number of Overall Staff']*100
merged_df['Number of Students'] = merged_df['Number of Students']*100
merged_df['STAAR Score in Math'] = merged_df['STAAR Score in Math']*100
merged_df['STAAR Score in Reading'] = merged_df['STAAR Score in Reading']*100
merged_df['Math%'] = merged_df['Learning Loss in Math']
merged_df['Reading%'] = merged_df['Learning Loss in Reading']
merged_df['Loss%'] = merged_df['Learning Loss']
#merged_df['Learning Loss in Math'] = merged_df['Learning Loss in Math']*100
#merged_df['Learning Loss in Reading'] = merged_df['Learning Loss in Reading']*100
#merged_df['Learning Loss'] = merged_df['Learning Loss']*100

In [70]:
merged_df['AVG_Math_2019']

0       0.84
1       0.86
2       0.82
3       0.74
4       0.86
        ... 
1008    0.88
1009    0.85
1010    0.87
1011    0.87
1012    0.85
Name: AVG_Math_2019, Length: 1013, dtype: float64

In [71]:
def lossfuncmath(row):
    if row['Learning Loss in Math']>0:
        val = 'Gain'
    elif row['Learning Loss in Math']<0:
        val = 'Loss'
    else:
        val = 'No Unexpected Change'
    return val

merged_df['Label Math'] = merged_df.apply(lossfuncmath, axis=1)

def lossfuncreading(row):
    if row['Learning Loss in Reading']>0:
        val = 'Gain'
    elif row['Learning Loss in Reading']<0:
        val = 'Loss'
    else:
        val = 'No Unexpected Change'
    return val

merged_df['Label Reading'] = merged_df.apply(lossfuncreading, axis=1)

def lossfunc(row):
    if row['Learning Loss']>0:
        val = 'Gain'
    elif row['Learning Loss']<0:
        val = 'Loss'
    else:
        val = 'No Unexpected Change'
    return val

merged_df['Label Overall'] = merged_df.apply(lossfunc, axis=1)

In [72]:
merged_df['Learning Loss in Math'].dtypes

dtype('int64')

In [73]:
merged_df.to_csv('Data/DistrictLevelstats.csv')

In [74]:
merged_df.dtypes.tail(10)

Learning Loss                int64
Number of Teachers         float64
Number of Overall Staff    float64
Number of Students         float64
Math%                        int64
Reading%                     int64
Loss%                        int64
Label Math                  object
Label Reading               object
Label Overall               object
dtype: object

In [75]:
merged_df['Number of Overall Staff'] = merged_df['Number of Overall Staff'].astype(int)
merged_df['Number of Students'] = merged_df['Number of Students'].astype(int)

In [76]:
merged_df['Number of Teachers'] = merged_df['Number of Teachers'].astype(int)
merged_df['STAAR Score in Math'] = merged_df['STAAR Score in Math'].astype(int)
merged_df['STAAR Score in Reading'] = merged_df['STAAR Score in Reading'].astype(int)

merged_df["Learning Loss in Math"] = merged_df["Learning Loss in Math"].astype(str)
merged_df["Learning Loss in Reading"] = merged_df["Learning Loss in Reading"].astype(str)
merged_df["Learning Loss"] = merged_df["Learning Loss"].astype(str)

In [77]:
#Creates a county map of Texas based off of the National Center on Education Statistics DataFrame
input_list = ['Number of Teachers',
              'Number of Overall Staff',
              'Number of Students',
              'STAAR Score in Math',
              'STAAR Score in Reading',
              'Learning Loss in Math',
              'Learning Loss in Reading',
              'Learning Loss']
              
input_list = sorted(input_list)
district_list = label_data['County Name'].values
district_list = sorted(district_list)
class MapDashboard(param.Parameterized):
    __ = param.Selector(default='Hays',objects=district_list,check_on_set=False)
  

    _ = param.ObjectSelector(default=input_list[0],objects=input_list)
   # @param.depends('_')
    def get_map(self):
        county_num = cdict[self.__]
        
        dis_df = merged_df.loc[merged_df['County #'] == county_num]
        data = dis_df[['NAME','County #','DISTRICT_N','% On Campus 10/30/20','% Tested Math G8 Diff','CARES ESSER I 20',
                    '% Black Diff','% Asian Diff','% Reduced-price Lunch Diff','Label Math',
              'Label Reading', 'Math%','Reading%','Loss%',
              'Label Overall',self._]]
        
        if self._ == 'Learning Loss in Math' or self._ == 'Learning Loss in Reading' or self._ == 'Learning Loss':
            fig = px.choropleth(data, geojson=dis_df, locations='DISTRICT_N', color= self._,
            color_continuous_scale=[(0.00, "#7f0e28"),  (0.33, "#7f0e28"),
                                    (0.33, "#E0D4CF"),  (0.66, "#E0D4CF"),
                                    (0.66, "#19204E"),  (1.00, "#19204E")],
            featureidkey='properties.DISTRICT_N',
            title =self.__ + ' County',
            custom_data = [data['NAME'],self._,
                       data['Label Math'],data['Label Reading'],data['Label Overall'],
                       data['Math%'],data['Reading%'],data['Loss%']],
            height=550, width=480
            )
            fig.update_layout(coloraxis_colorbar=dict(tickvals=[-0.7,0,0.7],ticktext=['-1','0','1']))
        else:
            minimum = -10
            maximum = 10
            fig = px.choropleth(data, geojson=dis_df, locations='DISTRICT_N', color= self._,
            color_continuous_scale='balance_r',
            range_color=(minimum,maximum),
            featureidkey='properties.DISTRICT_N',
            title =self.__ + ' County',
            custom_data = [data['NAME'],self._,
                       data['Label Math'],data['Label Reading'],data['Label Overall'],
                       data['Math%'],data['Reading%'],data['Loss%']],
            height=550, width=480
            )
            fig.update_layout(coloraxis_colorbar=dict(tickvals=[-8,-4,0,4,8],ticktext=['-10','-5','0','5','10']))

        fig.update_coloraxes(cmid=0,colorbar_orientation='h',colorbar_y=-0.1,colorbar_title_font_family='Times New Roman',
                                colorbar_title_font_size=24,colorbar_title_side='bottom',colorbar_thickness=4)
        
        
        
        
        if self._ == 'Learning Loss in Math':
            hovertemp = ''
            hovertemp += '%{customdata[0]}<br>'
            hovertemp += '(%{customdata[5]}) : %{customdata[2]}'
        elif self._ == 'Learning Loss in Reading':
            hovertemp = ''
            hovertemp += '%{customdata[0]}<br>'
            hovertemp += '(%{customdata[6]}) : %{customdata[3]}'
        elif self._ == 'Learning Loss':
            hovertemp = ''
            hovertemp += '%{customdata[0]}<br>'
            hovertemp += '(%{customdata[7]}) : %{customdata[4]}'
        else:
            hovertemp = ''
            hovertemp += '%{customdata[0]}<br>'
            hovertemp += '%{customdata[1]}% change in ' + self._
        fig.update_traces(hovertemplate=hovertemp)
        fig.update_geos(fitbounds='locations',visible=False)
        fig.update_layout(hoverlabel=dict(bgcolor='black',font_size=24,font_family="Times New Roman"),
        font=dict(family='times new roman',size=24,color='black'),
        title_x=.5,
        title_y=.97,                  
        margin={"r":10,"t":0,"l":0,"b":0})
        
        
        return fig
    def get_description(self):
        if self._ == 'STAAR Score in Math':
            description = pn.pane.HTML('The percent difference in Math STAAR scores between 2019 and 2021 for a district.',
                            style={'color':'black','font-size':'20px','width':'125%',
                                  'text-align':'left',
                                  'font-family':'Times New Roman'})
        elif self._ == 'STAAR Score in Reading':
            description = pn.pane.HTML('The percent difference in Reading STAAR scores between 2019 and 2021 for a district.',
                            style={'color':'black','font-size':'20px','width':'125%',
                                  'text-align':'left',
                                  'font-family':'Times New Roman'})
        elif self._ == 'Civilian Labor Force Difference (2021-2019)':
            description = pn.pane.HTML('This describes the difference in labor force between 2021 and 2019 for a district or county.',
                            style={'color':'black','font-size':'20px','width':'125%',
                                  'text-align':'left',
                                  'font-family':'Times New Roman'})
        elif self._ == 'Number of Teachers':
            description = pn.pane.HTML('The percent difference in Full-Time Teachers between 2019 and 2021 for a district.',
                            style={'color':'black','font-size':'20px','width':'125%',
                                  'text-align':'left',
                                  'font-family':'Times New Roman'})
        elif self._ == 'Number of Overall Staff':
            description = pn.pane.HTML('The percent difference in staff between 2019 and 2021 for a district.',
                            style={'color':'black','font-size':'20px','width':'125%',
                                  'text-align':'left',
                                  'font-family':'Times New Roman'})
        elif self._== 'Number of Students':
            description = pn.pane.HTML('The percent difference in students between 2019 and 2021 for a district.',
                            style={'color':'black','font-size':'20px','width':'125%',
                                  'text-align':'left',
                                  'font-family':'Times New Roman'})
        elif self._== 'Learning Loss in Math':
            description = pn.pane.HTML('This reflects whether a district experienced a loss (-1), a gain (+1), or no unexpected change (0) in Math STAAR scores from 2019 to 2021.',
                            style={'color':'black','font-size':'20px','width':'125%',
                                  'text-align':'left',
                                  'font-family':'Times New Roman'})
        elif self._== 'Learning Loss in Reading':
            description = pn.pane.HTML('This reflects whether a district experienced a loss (-1), a gain (+1), or no unexpected change (0) in Reading STAAR scores from 2019 to 2021.',
                            style={'color':'black','font-size':'20px','width':'125%',
                                  'text-align':'left',
                                  'font-family':'Times New Roman'})
        elif self._== 'Learning Loss':
            description = pn.pane.HTML('This reflects whether a district experienced a loss (-1), a gain (+1), or no unexpected change (0) in STAAR scores from 2019 to 2021.',
                            style={'color':'black','font-size':'20px','width':'125%',
                                  'text-align':'left',
                                  'font-family':'Times New Roman'})
        return description
    def panel(self):
        return pn.Row(
            pn.Column(
            pn.pane.HTML("""<h1>Select A County & Variable To View At The District Level </h1>""",
                              style={'color':'black','font-size':'14px','width':'100%',
                                     'text-align':'center',
                                     'font-family':'Times New Roman'}),
            pn.Param(self,width=550,background='#FFFFFF',
                     widgets={'__':{'widget_type':pn.widgets.AutocompleteInput,'placeholder':'Enter a county name...'}}
                              ),
         
            pn.Spacer(height=50),
            pn.pane.HTML("""<h1>What does this variable show us?</h1>""",
                            style={'color':'black','font-size':'14px','width':'100%',
                                  'text-align':'center',
                                  'font-family':'Times New Roman'}),
            pn.Spacer(height= 5),
            self.get_description
            #pn.Param(self, width=500,background='#FFFFFF'),
            ),
            pn.Spacer(width=150),
            self.get_map
        )
dashboard = MapDashboard(name='')
map_component2 = dashboard.panel()
map_component2

BokehModel(combine_events=True, render_bundle={'docs_json': {'1e67bb07-aed2-44fb-ae5f-5865b2554ba2': {'defs': …

# County Level Dashboard

In [78]:
#Panel HTML elements can take a 'style' parameter directly. 
#You could alternatively assign a css class as above and include the css code in raw_css
title_html_pane = pn.pane.HTML("""<h1>Learning Loss Recovery after COVID-19 Pandemic in Texas Public Schools</h1>""",
                              style={'color':'white','font-size':'20px','width':'145%',
                                     'background-color':'#6A5638','text-align':'center',
                                     'outline-style':'solid','outline-color':'#30231D',
                                     'outline-width':'thick','font-family':'Times New Roman'})
#datalab_img = pn.pane.PNG("C:/Users/Daniel Payan/Downloads/dataimg.png",height=220,width=410)
#txst_img = pn.pane.PNG("C:/Users/Daniel Payan/Downloads/txst1.png",width=410, height=220)
datalab_img = pn.pane.PNG("Data/Images/datalabimage.png",height=171,width=280)
txst_img = pn.pane.PNG("Data/Images/THRClogo.png",height=140,width=350)


title_bar = pn.Row(pn.Spacer(width=30),datalab_img,pn.Spacer(width=55),title_html_pane,pn.Spacer(width=310),txst_img)

content_bar = pn.Row(pn.Spacer(width=225),map_component1)

#Create a new gridspec with 14 columns and 12 rows
gspec = pn.GridSpec(ncols=14,nrows=6, background='#FFFFFF',sizing_mode='stretch_both')
#Place application elements in the grid, using pn.Spacer() for improved layout spacing and control
gspec[0, 7] = title_bar
gspec[1:6, 7] = content_bar
gspec.show()

Launching server at http://localhost:9507


# District Level Dashboard

In [79]:
#Panel HTML elements can take a 'style' parameter directly. 
#You could alternatively assign a css class as above and include the css code in raw_css
title_html_pane = pn.pane.HTML("""<h1>Learning Loss Recovery after COVID-19 Pandemic in Texas Public Schools</h1>""",
                              style={'color':'white','font-size':'20px','width':'145%',
                                     'background-color':'#6A5638','text-align':'center',
                                     'outline-style':'solid','outline-color':'#30231D',
                                     'outline-width':'thick','font-family':'Times New Roman'})
#datalab_img = pn.pane.PNG("C:/Users/Daniel Payan/Downloads/dataimg.png",height=220,width=410)
#txst_img = pn.pane.PNG("C:/Users/Daniel Payan/Downloads/txst1.png",width=410, height=220)
datalab_img = pn.pane.PNG("Data/Images/datalabimage.png",height=171,width=280)
txst_img = pn.pane.PNG("Data/Images/THRClogo.png",height=140,width=350)



title_bar = pn.Row(pn.Spacer(width=30),datalab_img,pn.Spacer(width=55),title_html_pane,pn.Spacer(width=310),txst_img)

content_bar = pn.Row(pn.Spacer(width=250),map_component2)

#Create a new gridspec with 14 columns and 12 rows
gspec = pn.GridSpec(ncols=14,nrows=6, background='#FFFFFF',sizing_mode='stretch_both')
#Place application elements in the grid, using pn.Spacer() for improved layout spacing and control
gspec[0, 7] = title_bar
gspec[1:6, 7] = content_bar
gspec.show()

Launching server at http://localhost:9508


# Heatmap & Histogram Dashboard

In [80]:
#Panel HTML elements can take a 'style' parameter directly. 
#You could alternatively assign a css class as above and include the css code in raw_css
title_html_pane = pn.pane.HTML("""<h1>Learning Loss Recovery after COVID-19 Pandemic in Texas Public Schools</h1>""",
                              style={'color':'white','font-size':'20px','width':'145%',
                                     'background-color':'#6A5638','text-align':'center',
                                     'outline-style':'solid','outline-color':'#30231D',
                                     'outline-width':'thick','font-family':'Times New Roman'})
#datalab_img = pn.pane.PNG("C:/Users/Daniel Payan/Downloads/dataimg.png",height=220,width=410)
#txst_img = pn.pane.PNG("C:/Users/Daniel Payan/Downloads/txst1.png",width=410, height=220)
datalab_img = pn.pane.PNG("Data/Images/datalabimage.png",height=171,width=280)
txst_img = pn.pane.PNG("Data/Images/THRClogo.png",height=140,width=350)



title_bar = pn.Row(pn.Spacer(width=30),datalab_img,pn.Spacer(width=55),title_html_pane,pn.Spacer(width=310),txst_img)

content_bar = pn.Row(pn.Spacer(width=100),loss_component1, pn.Spacer(width=100),loss_component2)

#Create a new gridspec with 14 columns and 12 rows
gspec = pn.GridSpec(ncols=14,nrows=6, background='#FFFFFF',sizing_mode='stretch_both')
#Place application elements in the grid, using pn.Spacer() for improved layout spacing and control
gspec[0, 7] = title_bar
gspec[2:6, 7] = content_bar
gspec.show()

Launching server at http://localhost:9510


# Definitions

In [81]:
#Panel HTML elements can take a 'style' parameter directly. 
#You could alternatively assign a css class as above and include the css code in raw_css
title_html_pane = pn.pane.HTML("""<h1>Learning Loss Recovery after COVID-19 Pandemic in Texas Public Schools</h1>""",
                              style={'color':'white','font-size':'20px','width':'145%',
                                     'background-color':'#6A5638','text-align':'center',
                                     'outline-style':'solid','outline-color':'#30231D',
                                     'outline-width':'thick','font-family':'Times New Roman'})
#datalab_img = pn.pane.PNG("C:/Users/Daniel Payan/Downloads/dataimg.png",height=220,width=410)
#txst_img = pn.pane.PNG("C:/Users/Daniel Payan/Downloads/txst1.png",width=410, height=220)
datalab_img = pn.pane.PNG("Data/Images/datalabimage.png",height=171,width=280)
txst_img = pn.pane.PNG("Data/Images/THRClogo.png",height=140,width=350)



title_bar = pn.Row(pn.Spacer(width=30),datalab_img,pn.Spacer(width=55),title_html_pane,pn.Spacer(width=310),txst_img)
index_pane = pn.pane.HTML("""<h1>Variable Definitions</h1>""", 
                          style={'color':'white','width':'165%','font-size':'20px','text-align':'center',    
                      #    'outline-style':'solid','outline-color':'#30231D',   
                      #    'outline-width':'thick',
                                 'background-color':'#501214',
                          'font-family':'Times New Roman',
                          'border':'10px solid black'})
table = pn.pane.HTML(
"""
<table style="width:100%">
<th style = 'font-size':'8px'>
  <tr>
    <th>Variable</th>
    <th>Reflects the percent difference of...</th>
  </tr>
  <tr>
    <td>STAAR Score in Math</td>
    <td>Math STAAR scores between 2019 and 2021 for a district/county.</td>
  </tr>
  <tr>
    <td>STAAR Score in Reading</td>
    <td>Reading STAAR scores between 2019 and 2021 for a district/county.</td>
  </tr>
  <tr>
    <td>Labor Force</td>
    <td>Labor force between 2019 and 2021 for a county.</td>
 </tr>
 <tr>
    <td>Number of Overall Staff</td>
    <td>Staff between 2019 and 2021 for a district/county.</td>
 </tr>
 <tr>
    <td>Number of Students</td>
    <td>Students between 2019 and 2021 for a district/county.</td>
 </tr>
 <tr>
    <td>Number of Teachers</td>
    <td>Full-Time Teachers between 2019 and 2021 for a district/county.</td>
 </tr>
 <tr>
    <th>******************************************************************</th>
    <th>*******************************************************************</th>
 </tr>
 <tr>
    <th>Variable</th>
    <th>Reflects whether a district experienced a loss(-1), gain(+1),<br> or no unexpected change(0) in...</th>
  </tr>
  <tr>
    <td>Learning Loss in Math</td>
    <td>Learning Loss in Math STAAR scores from 2019 to 2021.</td>
  </tr>
  <tr>
    <td>Learning Loss in Reading</td>
    <td>Learning Loss in Reading STAAR scores from 2019 to 2021.</td>
  </tr>
  <tr>
    <td>Learning Loss</td>
    <td>Learning Loss in STAAR scores from 2019 to 2021.</td>
  </tr>
</table>
""", style={'width':'165%','font-size':'20px','text-align':'center',    
            #'outline-style':'solid','outline-color':'#30231D',   
           # 'outline-width':'thick',
            'font-family':'Times New Roman',
            'border':'4px solid black'})
content_bar = pn.Row(pn.Spacer(width=150),
                     pn.Column(index_pane,table))

#Create a new gridspec with 14 columns and 12 rows
gspec = pn.GridSpec(ncols=14,nrows=12, background='#FFFFFF',sizing_mode='stretch_both')
#Place application elements in the grid, using pn.Spacer() for improved layout spacing and control
gspec[0, 7] = title_bar
#gspec[1:6, 0] = pn.Spacer()
#gspec[1:6, 5] = pn.Spacer()
gspec[2:6, 7] = content_bar
gspec.show()

Launching server at http://localhost:9511


#Panel HTML elements can take a 'style' parameter directly. 
#You could alternatively assign a css class as above and include the css code in raw_css
title_html_pane = pn.pane.HTML("""<h1>Learning Loss Recovery after COVID-19 Pandemic in Texas Public Schools</h1>""",
                              style={'color':'white','font-size':'26px','width':'160%',
                                     'background-color':'#6A5638','text-align':'center',
                                     'outline-style':'solid','outline-color':'#30231D',
                                     'outline-width':'thick','font-family':'Times New Roman'})
#datalab_img = pn.pane.PNG("C:/Users/Daniel Payan/Downloads/dataimg.png",height=220,width=410)
#txst_img = pn.pane.PNG("C:/Users/Daniel Payan/Downloads/txst1.png",width=410, height=220)
datalab_img = pn.pane.PNG("Data/Images/datalabimage.png",height=220,width=410)
txst_img = pn.pane.JPG("Data/Images/txstlogo.jpg",width=410, height=220)


title_bar = pn.Row(pn.Spacer(width=70),datalab_img,pn.Spacer(width=150),title_html_pane,pn.Spacer(width=550),txst_img,pn.Spacer(width=15))

content_bar = pn.Row(pn.Spacer(width=305),map_component2)

#Create a new gridspec with 14 columns and 12 rows
gspec = pn.GridSpec(ncols=14,nrows=12, background='#FFFFFF',sizing_mode='stretch_both')
#Place application elements in the grid, using pn.Spacer() for improved layout spacing and control
gspec[0, 7] = title_bar
#gspec[1:6, 0] = pn.Spacer()
#gspec[1:6, 5] = pn.Spacer()
gspec[2:6, 7] = content_bar
gspec.show()